In [20]:
import random
import numpy as np
import math
import pandas as pd

In [21]:
with open('rowZip.csv', 'r') as file:
    rows = file.readlines()
rows = [[int(st) for st in row.split(',')] for row in rows]
rows[0]

[12, 42, 170, 186, 216, 439, 461, 476, 493, 540, 547, 566, 605, 755, 794]

In [22]:
with open('colZip.csv', 'r') as file:
    cols = file.readlines()
cols = [[int(st) for st in col.split(',')] for col in cols]
cols[0]

[0,
 4,
 5,
 8,
 10,
 13,
 18,
 19,
 23,
 26,
 32,
 35,
 36,
 37,
 51,
 58,
 60,
 66,
 82,
 99,
 110,
 136,
 253]

def user_ranstr(num):
    userID = ''
    H = 'AB'
    userID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        userID += random.choice(H)
    return userID


def item_ranstr(num):
    itemID = ''
    H = 'def'
    itemID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        itemID += random.choice(H)
    return itemID

In [23]:
def initial_dataset(n,num,num2):
    #n: grid width, num: length of user/item ID , num2: max_bicluster_width
    bicluster = []
    for i in range(n*n):
        '''num_user = random.randrange(num2)
        num_item = random.randrange(num2)
        userID = []
        itemID = []
        j = 0
        k = 0
        while j < num_user:
            try:
                user = user_ranstr(num)
                userID.index(user)
            except:
                userID.append(user)
                j= j + 1 
        while k < num_item:
            try:
                item = item_ranstr(num)
                itemID.index(item)
            except:
                itemID.append(item)
                k = k + 1'''
        userID = rows[i]
        itemID = cols[i]
        single_set = [userID,itemID]
        bicluster.append(single_set)
    return bicluster

In [24]:
def reward_func(set1,set2):   
    intersection = 0
    for i in range(len(set2)):
        try:
            set1.index(set2[i])
            intersection = intersection + 1
     #       print(set2[i])
        except:
            pass 
   ## print("交:",intersection)
   # print("並:",(len(set1)+len(set2)-intersection))
    try:
        reward = intersection/(len(set1)+len(set2)-intersection)
    except:
        reward = 1
    return round(reward,3)

In [25]:
def gen_gridworld(width, data):
    left_num = width*width
    gridworld = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            num = random.randrange(left_num)
            gridworld[i][j] = data[num]
            data.remove( data[num])
            left_num = left_num - 1
    return gridworld

In [26]:
def gen_reward_grid(gridworld,width):
    grid_reward = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            reward=[]
            #print("i=",i,"j=",j)
            if j%width == width-1:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)       
            elif j%width == 0:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
            else: 
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
            grid_reward[i][j] = reward
    return grid_reward

In [27]:
def total_reward(grid_reward):
    return np.sum(grid_reward)

In [28]:
def hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta,iteration):
    i = random.randrange(width)
    j = random.randrange(width)
    i2 = random.randrange(width)
    j2 = random.randrange(width)
    data_grid[i][j], data_grid[i2][j2] =  data_grid[i2][j2], data_grid[i][j]
    reward_update(reward_grid,data_grid,i,j, width)
    reward_update(reward_grid,data_grid,i2,j2, width)
    new_tot_reward = total_reward(reward_grid)
    P_k = 1.0-beta*alpha**iteration
    #print("P_k:",P_k)
    backfire = random.random()
   # print("backfire:",backfire)
    T = 10000000
    flag = (iteration % T == 0)
    if P_k < backfire:
        if flag:
            print("backfire,old:",tot_reward," new:",new_tot_reward)
        return new_tot_reward, 1
    else: 
        if new_tot_reward > tot_reward:
            if flag:
                print("update:",new_tot_reward)
            return new_tot_reward, 1
        else:
            if flag:
                print("non_update:",tot_reward," new:F",new_tot_reward)
            data_grid[i][j],  data_grid[i2][j2] =   data_grid[i2][j2],  data_grid[i][j]
            reward_update(reward_grid,data_grid,i,j, width)
            reward_update(reward_grid,data_grid,i2,j2, width)
            return tot_reward, 0

In [29]:
def reward_update(grid_reward,gridworld,x,y, width):
    x_update = [x-1,x,x+1,x,x]
    y_update = [y,y,y,y-1,y+1]
    for i, j in zip(x_update, y_update):
        reward=[]
        #print("i=",i,"j=",j)
        if i < 0 or i > width-1 or j < 0 or j> width-1:
           # print("i=",i,"j=",j)
            continue
        if j%width == width-1:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)       
        elif j%width == 0:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
        else: 
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
        grid_reward[i][j] = reward

In [30]:
width = 20
id_length = 2
max_bicluster_width = 20
beta = 0.4
stop_value = 10000

data = initial_dataset(width,id_length,max_bicluster_width)
data_grid = gen_gridworld(width, data)
reward_grid = gen_reward_grid(data_grid,width)
tot_reward = total_reward(reward_grid)
print(tot_reward)
stop = 0
alpha = 0.99999
iteration = 0

679.386


In [31]:
stop_value

10000

In [32]:
while True:
    iteration = iteration + 1
    tot_reward, update_bool = hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta, iteration)
    P_k = 1.0-beta*alpha**iteration
    if update_bool == 1:
        stop = 0
    else:
        stop = stop+1
    if iteration % 10000 == 0:
        print(P_k, tot_reward)
    if 1 - P_k < 1e-7:
        break
    #print("real: ", total_reward(gen_reward_grid(data_grid,width)))
print(tot_reward)

0.6380652137540963 726.488
0.67250802626283 737.982
0.7036731562204706 731.6779999999999
0.7318725178443317 741.304
0.7573883426483397 732.68
0.780476004139156 737.742
0.8013665737056469 756.27
0.8202691332821546 759.814
0.8373728679317141 775.71
0.8528489592927013 778.712
0.866852298839354 759.5819999999999
0.8795230381032203 794.564
0.8909879913708328 792.8199999999999
0.9013619048964144 796.2259999999999
0.9107486053324516 791.4839999999999
0.9192420388721302 795.076
0.9269272115038236 826.6759999999999
0.9338810397881107 823.3799999999999
0.9401731206722711 835.2339999999999
0.9458664280468974 840.116
0.9510179430160663 829.2459999999999
0.9556792241890882 859.674
0.9598969237015627 863.53
0.9637132541303047 872.212
0.967166410975231 882.2479999999998
0.970290954936586 892.182
0.9731181578135083 912.9759999999999
0.97567631548584 904.7339999999999
0.9779910311116368 874.674
0.9800854713747438 902.2379999999999
0.981980598347075 908.87
0.9836953792861737 938.588
0.9852469764678018 9

In [33]:
total_reward(gen_reward_grid(data_grid,width))

1202.8899999999999

In [34]:
data_grid[0][1]

[[12, 42, 69, 170, 186, 216, 320, 439, 476, 493, 540, 547, 571, 625, 755],
 [0,
  2,
  3,
  4,
  5,
  8,
  9,
  11,
  18,
  23,
  26,
  35,
  37,
  39,
  41,
  51,
  58,
  66,
  83,
  89,
  99,
  109,
  136]]

In [35]:
import pickle

f1 = open("dic", "rb")
f2 = open("dic_inv", "rb")
dic = pickle.load(f1)
dic_inv = pickle.load(f2)
f1.close()
f2.close()

In [36]:
real_grid = [[[data_grid[k][j][0], [dic_inv[i] for i in data_grid[j][k][1]]] for j in range(width)] for k in range(width)]

In [37]:
f = open("grid3", "wb")
pickle.dump(real_grid, f)
f.close()

In [38]:
f = open("grid", "rb")
pickle.load(f) == real_grid
f.close()

In [39]:
real_grid

[[[[12, 42, 69, 170, 186, 216, 320, 439, 461, 476, 493, 540, 547, 571, 625],
   [356,
    318,
    593,
    2571,
    480,
    110,
    1196,
    1198,
    1270,
    2028,
    4306,
    3578,
    1580,
    4226,
    1265,
    1291,
    1240,
    6377,
    2716,
    8961,
    3147,
    5418,
    2115]],
  [[12, 42, 69, 170, 186, 216, 320, 439, 476, 493, 540, 547, 571, 625, 755],
   [356,
    318,
    593,
    2571,
    480,
    110,
    1196,
    1198,
    1270,
    2028,
    4306,
    3578,
    1580,
    4226,
    1291,
    1240,
    6377,
    2716,
    4963,
    8961,
    3147,
    5418,
    2115]],
  [[12, 42, 108, 170, 186, 216, 320, 430, 439, 493, 540, 547, 641, 755, 794],
   [356,
    318,
    593,
    2571,
    480,
    110,
    1196,
    1198,
    1270,
    2028,
    2762,
    4306,
    3578,
    1580,
    4226,
    1291,
    1240,
    1097,
    6377,
    2716,
    8961,
    3147,
    5418,
    2115]],
  [[12, 42, 68, 108, 170, 186, 216, 320, 430, 439, 493, 540, 547, 755, 794],
